In [159]:
import tensorflow as tf
import numpy as np
from keras.utils import load_img, img_to_array
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
from matplotlib import pyplot as plt


In [160]:
model = tf.keras.models.load_model('m-and-m.h5')

In [161]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 49, 49, 16)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 23, 23, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 16928)             0         
                                                                 
 dense_4 (Dense)             (None, 512)              

In [162]:
img = load_img('./test_images/IMG_1312.jpg', target_size=(100, 100))
x = img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
print(model.predict(images, batch_size=10))

1/1 [==============================] - 0s 113ms/step
[[0.9999714]]


2023-01-04 11:36:30.287248: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [170]:
imgHeight = 1037
imgWidth = 1037
stepSize = 20
img = load_img('./test_images/input.jpg', target_size=(imgWidth, imgHeight))
x = img_to_array(img)
x /= 255
x.flags["WRITEABLE"] = False

# imgWidth/stepSize = 189
# imgHeight/stepSize = 189
boxes = np.array([])
fits = np.array([])

for i in range(0, imgWidth - 100, stepSize):
    for j in range(0, imgHeight - 100, stepSize):
        topI = i + 100
        topJ = j + 100
        croppedImg = x[i:topI,j:topJ,:]
        expanded = np.expand_dims(croppedImg, axis=0)
        images = np.vstack([expanded])
        v = model.predict(images, batch_size=10, verbose=0)
        value = v[0]
        I = 0
        J = 0
        if(i!=0):
            I = int(i/stepSize)
        if(j!=0):
            J = int(j/stepSize)
        if(value[0] > 0.99):
            boxes = np.append(boxes, [i, j, topI, topJ])
            fits = np.append(fits, value[0])
            print([i, j, topI, topJ])
            print(value[0])


[0, 860, 100, 960]
0.9911992
[20, 860, 120, 960]
0.9914601
[40, 860, 140, 960]
0.9903598
[60, 880, 160, 980]
0.9904757
[80, 0, 180, 100]
0.9909538
[80, 880, 180, 980]
0.99096936
[100, 0, 200, 100]
0.99133885
[100, 880, 200, 980]
0.9917264
[120, 0, 220, 100]
0.9902091
[120, 880, 220, 980]
0.9916545
[140, 880, 240, 980]
0.9907449
[160, 900, 260, 1000]
0.9910683
[180, 900, 280, 1000]
0.990379
[200, 240, 300, 340]
0.99428225
[200, 260, 300, 360]
0.995456
[220, 260, 320, 360]
0.99089855
[220, 900, 320, 1000]
0.9901575
[300, 260, 400, 360]
0.99449843
[300, 280, 400, 380]
0.99619544
[300, 920, 400, 1020]
0.99042976
[320, 240, 420, 340]
0.9940572
[320, 260, 420, 360]
0.998582
[320, 280, 420, 380]
0.9983371
[320, 300, 420, 400]
0.99159795
[320, 680, 420, 780]
0.9919512
[320, 700, 420, 800]
0.9959401
[320, 720, 420, 820]
0.9904987
[320, 920, 420, 1020]
0.9906929
[340, 240, 440, 340]
0.99715865
[340, 260, 440, 360]
0.9988436
[340, 280, 440, 380]
0.9967918
[340, 440, 440, 540]
0.99177355
[340, 460

In [171]:
length = len(boxes)
newLength = round(length/4)
boxes = np.reshape(boxes, (newLength, 4))


In [172]:
print(boxes.shape)

(198, 4)


In [173]:
#@title Plot Utilities for Bounding Boxes [RUN ME]
#This is from the advanced computer vision with tensorflow course on coursera
# https://www.coursera.org/learn/advanced-computer-vision-with-tensorflow/home/info

im_width = 1037
im_height = 1037
use_normalized_coordinates = True

def draw_bounding_boxes_on_image_array(image,
                                       boxes,
                                       color=[],
                                       thickness=1,
                                       display_str_list=()):
  """Draws bounding boxes on image (numpy array).
  Args:
    image: a numpy array object.
    boxes: a 2 dimensional numpy array of [N, 4]: (ymin, xmin, ymax, xmax).
           The coordinates are in normalized format between [0, 1].
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list_list: a list of strings for each bounding box.
  Raises:
    ValueError: if boxes is not a [N, 4] array
  """
  image_pil = PIL.Image.fromarray(image)
  rgbimg = PIL.Image.new("RGBA", image_pil.size)
  rgbimg.paste(image_pil)
  draw_bounding_boxes_on_image(rgbimg, boxes, color, thickness,
                               display_str_list)
  return np.array(rgbimg)
  

def draw_bounding_boxes_on_image(image,
                                 boxes,
                                 color=[],
                                 thickness=1,
                                 display_str_list=()):
  """Draws bounding boxes on image.
  Args:
    image: a PIL.Image object.
    boxes: a 2 dimensional numpy array of [N, 4]: (ymin, xmin, ymax, xmax).
           The coordinates are in normalized format between [0, 1].
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list: a list of strings for each bounding box.
                           
  Raises:
    ValueError: if boxes is not a [N, 4] array
  """
  boxes_shape = boxes.shape
  if not boxes_shape:
    return
  if len(boxes_shape) != 2 or boxes_shape[1] != 4:
    raise ValueError('Input must be of size [N, 4]')
  for i in range(boxes_shape[0]):
    draw_bounding_box_on_image(image, boxes[i, 1], boxes[i, 0], boxes[i, 3],
                               boxes[i, 2], color, thickness)
        
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color='red',
                               thickness=20,
                               display_str=None,
                               use_normalized_coordinates=False):
  """Adds a bounding box to an image.
  Bounding box coordinates can be specified in either absolute (pixel) or
  normalized coordinates by setting the use_normalized_coordinates argument.
  Args:
    image: a PIL.Image object.
    ymin: ymin of bounding box.
    xmin: xmin of bounding box.
    ymax: ymax of bounding box.
    xmax: xmax of bounding box.
    color: color to draw bounding box. Default is red.
    thickness: line thickness. Default value is 4.
    display_str_list: string to display in box
    use_normalized_coordinates: If True (default), treat coordinates
      ymin, xmin, ymax, xmax as relative to the image.  Otherwise treat
      coordinates as absolute.
  """
  draw = PIL.ImageDraw.Draw(image)
  im_width, im_height = image.size
  if use_normalized_coordinates:
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
  else:
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
  draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)
  


In [174]:
img_to_draw = draw_bounding_boxes_on_image_array(image=np.uint8(x*255), boxes=boxes, color='green')
image_pil = PIL.Image.fromarray(np.uint8(img_to_draw))

In [175]:
image_pil.save("output.png")